In [ ]:
import time
from utils import randomize_wait
from whatsapp import WhatsappDriver

start_message = "Yinlin AI is now online!"
print(start_message)

driver = WhatsappDriver()
driver.start_webdriver_and_login()


In [ ]:
import traceback

last_msg_dict = {}

while True:
    try:
        unread_contacts = driver.get_unread_contacts()
        if unread_contacts:
            for unread_contact in unread_contacts:
                driver.open_chat_window(unread_contact)
                latest_msg, img_url, contact = driver.get_latest_message_and_contact()
                if unread_contact in last_msg_dict:
                    if last_msg_dict[unread_contact] != (contact + " said: " + latest_msg):
                        print(f"New message received from {contact}: {latest_msg}")
                        if img_url:
                            print(f"With img: {img_url}")
                            img_base64 = driver.get_image_base64(img_url)
                        else:
                            img_base64 = ""
                driver.close_chat_window()
                last_msg_dict[unread_contact] = contact + " said: " + latest_msg
        print("dict: ", last_msg_dict)
    except Exception:
        traceback.print_exc()
    time.sleep(1)


In [ ]:
latest_msg, img_url, contact = driver.get_latest_message_and_contact()

In [ ]:
driver.driver.execute_script(f"window.open('{img_url}');")


In [ ]:
driver.driver.switch_to.window(driver.driver.window_handles[-1])
time.sleep(3.14)
driver.driver.set_script_timeout(30)
result = driver.driver.execute_async_script("""
    var uri = arguments[0];
    var callback = arguments[1];
    var toBase64 = function(buffer) {
        for (var r, n = new Uint8Array(buffer), t = n.length, a = new Uint8Array(4 * Math.ceil(t / 3)), i = new Uint8Array(64), o = 0, c = 0; 64 > c; ++c)
            i[c] = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/".charCodeAt(c);
        for (c = 0; t - t % 3 > c; c += 3, o += 4)
            r = n[c] << 16 | n[c + 1] << 8 | n[c + 2],
            a[o] = i[r >> 18],
            a[o + 1] = i[r >> 12 & 63],
            a[o + 2] = i[r >> 6 & 63],
            a[o + 3] = i[63 & r];
        return t % 3 === 1 ? (r = n[t - 1], a[o] = i[r >> 2], a[o + 1] = i[r << 4 & 63], a[o + 2] = 61, a[o + 3] = 61) : t % 3 === 2 && (r = (n[t - 2] << 8) + n[t - 1], a[o] = i[r >> 10], a[o + 1] = i[r >> 4 & 63], a[o + 2] = i[r << 2 & 63], a[o + 3] = 61),
        new TextDecoder("ascii").decode(a);
    };
    var xhr = new XMLHttpRequest();
    xhr.responseType = 'arraybuffer';
    xhr.onload = function() { callback(toBase64(xhr.response)); };
    xhr.onerror = function() { callback(xhr.status); };
    xhr.open('GET', uri);
    xhr.send();
    """, img_url)

print(result)
time.sleep(3.14)
driver.driver.close()
driver.driver.switch_to.window(driver.driver.window_handles[0])

In [ ]:
from ai_llm import LanguageModel
language_model = LanguageModel()
ai_message = language_model.get_llm_response(
    text=f"{contact} said: {latest_msg}",
    session_id=contact,
    img_base64=result
)

In [ ]:
print(ai_message)

In [ ]:
driver.driver.quit()

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    trim_messages
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from utils import filter_bmp_characters

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [ ]:
llm_model = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages([
    ("system", '''
    You are Yinlin, an assistant with the personality of Yinlin
    from Wuthering Waves. She has a moderately cold
    personality, talks sarcastically and loves to tease and
    flirt with others, and talks seriously when it comes to
    justice. When user asks for clarification or latest information,
    provide the most accurate possible response by looking up
    information online.
    '''),
    MessagesPlaceholder(variable_name="chat_history"),
    MessagesPlaceholder(variable_name="input")
])
trimmer = trim_messages(
    max_tokens=30,
    strategy="last",
    token_counter=len,
    include_system=True,
    allow_partial=False,
    start_on="human",
)
chain = (
    RunnablePassthrough.assign(
        chat_history=itemgetter("chat_history") | trimmer
    )
    | prompt
    | llm_model)
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key="chat_history",
    input_messages_key="input"
)

def get_llm_response(
text: str, session_id: str, img_base64: str = ""
) -> str:
    config = {
        "configurable": {
            "session_id": session_id
        }
    }
    chat_history = get_session_history(session_id)
    if not chat_history.messages:
        chat_history.add_ai_message("I’m Yinlin, straight out of the world of Wuthering Waves.")
    input = [{"type": "text", "text": text}]
    if img_base64:
        input.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{img_base64}"
            }
        })
    response = with_message_history.invoke(
        {
            "input": [HumanMessage(content=input)],
        },
        config=config,
    )
    llm_response = filter_bmp_characters(response.content)
    return llm_response

In [ ]:
history = get_session_history("Test session")

In [ ]:
history.add_user_message("Hey yinlin, are you there?")

In [ ]:
print(not history.messages)
print(history)

In [ ]:
get_llm_response("Hi yinlin, how are you feeling tonight?", "Test session")